In [1]:
# Import necessary libraries for creating an agent-based conversation system
# This includes AsyncOpenAI client, Semantic Kernel components for agents,
# and utilities for managing chat conversations and termination strategies
import asyncio
import os
import asyncio

from typing import Annotated
from openai import AsyncOpenAI


from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)

from semantic_kernel.functions import KernelFunctionFromPrompt

from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

In [2]:
# Function to create a Semantic Kernel instance with OpenAI chat completion service
# This function initializes the kernel and configures it with the appropriate AI service
# Parameters:
#   service_id: A string identifier for the service being created
# Returns:
#   A configured Semantic Kernel instance ready for agent operations
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    # Create a new Kernel instance
    kernel = Kernel()
    # Set default service ID for the kernel
    service_id="agent"

    # Initialize the AsyncOpenAI client with Azure OpenAI endpoint
    # Uses GitHub token for authentication and Azure Inference API as base URL
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    
    # Add OpenAI chat completion service to the kernel
    # Configures the service with the specified model and client
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id=service_id
        )
    )

    # Return the configured kernel
    return kernel

In [3]:
# This function implements the main conversation flow between a hotel concierge, 
# a front desk agent, and a user seeking travel recommendations
# It demonstrates how to:
# 1. Set up multiple agents with specific roles and instructions
# 2. Create selection and termination strategies to control conversation flow
# 3. Process user input and manage a multi-agent conversation

# Define the main asynchronous function that orchestrates the conversation between agents
async def main():
    REVIEWER_NAME = "Concierge"
    REVIEWER_INSTRUCTIONS = """
    You are an are hotel concierge who has opinions about providing the most local and authetic experiences for travelers.
    The goal is to determine if the front desk travel agent has reccommended the best non-touristy experience for a travler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """
    agent_reviewer = ChatCompletionAgent(
        service_id="concierge",
        kernel=_create_kernel_with_chat_completion("concierge"),
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    FRONTDESK_NAME = "FrontDesk"
    FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activites and locations for a traveler to visit.
    Only provide a single recomendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """
    agent_writer = ChatCompletionAgent(
        service_id="frontdesk",
        kernel=_create_kernel_with_chat_completion("frontdesk"),
        name=FRONTDESK_NAME,
        instructions=FRONTDESK_INSTRUCTIONS,
    )

    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt="""
        Determine if the reccomendation is good.  If so, respond with a single word: yes

        History:
        {{$history}}
        """,
    )

    selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
        Determine which participant takes the next turn in a conversation based on the the most recent participant.
        State only the name of the participant to take the next turn.
        No participant should take more than one turn in a row.
        
        Choose only from these participants:
        - {REVIEWER_NAME}
        - {FRONTDESK_NAME}
        
        Always follow these rules when selecting the next participant, each conversation should be at least 4 turns:
        - After user input, it is {FRONTDESK_NAME}'s turn.
        - After {FRONTDESK_NAME} replies, it is {REVIEWER_NAME}'s turn.
        - After {REVIEWER_NAME} provides feedback, it is {FRONTDESK_NAME}'s turn.

        History:
        {{{{$history}}}}
        """,
    )

    chat = AgentGroupChat(
        agents=[agent_writer, agent_reviewer],
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_reviewer],
            function=termination_function,
            kernel=_create_kernel_with_chat_completion("termination"),
            result_parser=lambda result: str(result.value[0]).lower() == "yes",
            history_variable_name="history",
            maximum_iterations=10,
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion("selection"),
            result_parser=lambda result: str(
                result.value[0]) if result.value is not None else FRONTDESK_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
    )

    input = "i would like to go to Seoul, South Korea."

    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input))
    print(f"# User: '{input}'")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")

    print(f"# IS COMPLETE: {chat.is_complete}")

await main()

# User: 'i would like to go to Seoul, South Korea.'
# Agent - FrontDesk: 'Visit Gyeongbokgung Palace for a rich cultural experience and stunning architecture.'
# Agent - Concierge: 'While Gyeongbokgung Palace is indeed an iconic site, it can be quite touristy and crowded. To refine this recommendation, consider suggesting a lesser-known historical site or a more intimate neighborhood where travelers can experience local culture, perhaps through traditional markets, local eateries, or community events. Highlighting hidden gems will provide a more authentic and memorable experience.'
# IS COMPLETE: True
